Need to be aware of spaces when defining variables, recall spaces are used to separate options/flags, so doing `foo=bar` will work, whereas `foo = bar` will return an error (it is interpreted as calling the `foo` program, with `=` as the first argument and `bar` as the second argument)

We can put functions into files, execute and load it into the shell by `source`

file `mcd.sh`:
```
mcd() {
    mkdir "$1"
    cd "$1"
   }
```
sourcing it in shell:
`source mcd.sh`

`$_`: last argument of the previous command, this can be handy at times, for example creating a directory and changing directory to it:
```
mkdir test
cd $_
```

`$@`: expands to all arguments, handy when we don't know how many arguments there will be beforehand, example: `for file in "$@"; do`

`$#`: number of arguments

`$$`: process id

`$?`: error message of the previous command, if `echo $?` returned 0, it meant the previous command executed successfully

`$0`: name of the script

`$1`~`$9`: first to the 9th command

Another command that might come in handy is `!!`, for example, if we were to do something but was denied permission, then we can do:

```mkdir /directory/with/no/permission
sudo !!```

The `!!` will be replaced with the previous command

Getting the output of a command into a variable:

In [2]:
!foo=$(pwd)
!echo "we are in $(pwd)"

'foo' is not recognized as an internal or external command,
operable program or batch file.


"we are in $(pwd)"


### Process substitution:
`<(some_command)` makes the input or output of a command to appear as a file, this can come in handy for commands that doesn't take input from stdin but from files, for example, the following command concatenates the file names in the current and parent directory and prints it out on the terminal:

In [3]:
!cat <(ls) <(ls ..)

The system cannot find the file specified.


Example used in class:
```
for file in "$@"; do
    grep foobar "$file" > dev/null 2> /dev/null 
    if [[ "$?" -ne 0 ]]; then 
        echo "File $file does not have any foobar
    fi
done
```
Note: the line `grep foobar "$file" > dev/null 2> /dev/null` meant we are not interested in the result from stdout and stderr, so we dump it to /dev/null, 2> is for stderr

### Globbing:
wildcard matching with `*`, we can also use `$` for a one character wildcard matching, for example, if we have directories called `project1`, `project2` and `project42`, and we only want the projects with single digits, then we do `project$`

We can also use `{}`, for example, if we want to create files `foo1, foo2, foo10`, instead of typing all file names 1 by 1, we can just do `touch foo{1,2,10}`. If we do something like `touch project{1,2}/testP{1,2,3}.py`, then the shell will expand everything inside the curly braces into its cartesian product. We can also do things like `touch {foo, bar}/{a...j}`, in which case, the shell will create files `a` to `j` under the directory `foo` and `bar`

### Shebang
In the exercise from lecture 1, we already knew that #! will specify the interpreter directory, so that's why in Python scripts, we commonly see the first line as something similar to `#!usr/local/bin/python`, but what if we don't know where the `python` interpreter is, then we can try 

`#!usr/bin/env python`

instead, in this case, we make use of the `env` variable, and attempt to search for `python` in the `env`

### Shellcheck
Shellcheck helps check your file, finds if there are any errors and gives warning, usage:

`shellcheck some_file`

### tldr
Last lecture, we learned about using `man` to access the manual, but sometimes the manual is very long, and instead there is this other program called `tldr` that we can download, and it gives more succint instructions on how to run a program, usage:

`tldr some_command`

### How to find files
`find`, example: 

`find . -name src -type d`, `.` means the current directory, `d` means we want to find directories

`find . -path **/yest/*.py`, `**` means some number of folders

`find . -mtime -1`, `-mtime` stands for the the time the file was modified (in the unit of days)

We can also execute commands after we find the files:

`find . -name *.tmp -exec rm {} \;`

### More on grep
We can search recursively through a directory with `grep` using the `-R` flag, a potential use case for this is when we have multiple code files under a directory, and we want to find which file contained the function that we wrote 2 months ago, then running the `grep -R function_name .` command can help us with it.

There is also another similar library called `ripgrep` that has better syntax highlighting, more flags...., for example:

`rg -u --files-without-match "^#/!" -t sh`, `-u` means don't ignore hidden files, and the command finds all the `sh` files that doesn't have a shebang.

### Finding commands
Ctrl+R: enters recall mode to search for commands that we have previous used, for example, if we want to find all the `grep` commands that we have used befor, then Ctrl+R, `grep` and keeping hitting the uparrow key until we find what we want. 

We can also use the `history` command, to start from the history of the first command, do `history 1`, and then we can further pipe this to perhaps `grep` to refine our search, such as `history 1 | grep grep`